In [4]:
import numpy as np

np.random.seed(123)
X = np.load('../generated_games/features-40k.npy')
Y = np.load('../generated_games/labels-40k.npy')
print("features", X.shape)
print("labels", Y.shape)

samples = X.shape[0]
size = 9
input_shape = (size, size, 1)

X = X.reshape(samples, size, size, 1)

train_samples = int(0.9 * samples)
X_train, X_test = X[:train_samples], X[train_samples:]
Y_train, Y_test = Y[:train_samples], Y[train_samples:]
print("X_train", X_train.shape)
print("Y_train", Y_train.shape)
print("X_test", X_test.shape)
print("Y_test", Y_test.shape)

features (41439, 1, 9, 9)
labels (41439, 81)
X_train (37295, 9, 9, 1)
Y_train (37295, 81)
X_test (4144, 9, 9, 1)
Y_test (4144, 81)


In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input

model = Sequential()
model.add(Input(shape=input_shape))
model.add(Conv2D(filters=48,
                 kernel_size=(3, 3),
                 padding='same',
                 activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Conv2D(48, (3, 3),
                 padding='same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.5))
model.add(Flatten()) 
model.add(Dense(512, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(size * size, activation='softmax')) 
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 9, 9, 48)       │           480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 9, 9, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 9, 9, 48)       │        20,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 4, 4, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 4, 4, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │       393,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 81)             │        41,553 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 456,545 (1.74 MB)

 Trainable params: 456,545 (1.74 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
import time

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

# Record the start time
start_time = time.time()
model.fit(X_train, Y_train,
          batch_size=64,
          epochs=200,
          verbose=1,
          validation_data=(X_test, Y_test))
# Record the end time
end_time = time.time()
# Calculate and display the wall time
print(f"Wall time: {end_time - start_time:.6f} seconds")

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/200
583/583 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.0158 - loss: 4.3965 - val_accuracy: 0.0232 - val_loss: 4.3172
Epoch 2/200
583/583 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.0212 - loss: 4.3194 - val_accuracy: 0.0236 - val_loss: 4.2739
Epoch 3/200
583/583 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.0240 - loss: 4.2901 - val_accuracy: 0.0253 - val_loss: 4.2567
Epoch 4/200
583/583 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.0236 - loss: 4.2738 - val_accuracy: 0.0256 - val_loss: 4.2387
Epoch 5/200
583/583 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.0243 - loss: 4.2517 - val_accuracy: 0.0246 - val_loss: 4.2179
Epoch 6/200
583/583 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.0235 - loss: 4.2328 - val_accuracy: 0.0241 - val_loss: 4.1955
Epoch 7/200
583/583 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.0250 - loss: 4.2064 - val_accuracy: 0.0234 - val_loss: 4.1643
Epoch 8/200
583/583 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.0252 - loss: 4.1746 - val_accu

In [7]:
test_board = np.array([[
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 1, -1,  1, -1, 0, 0, 0, 0,
    0, 1, -1,  1, -1, 0, 0, 0, 0,
    0, 0,  1, -1,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
]]).reshape(1, 9, 9, 1)
move_probs = model.predict(test_board)[0]
i = 0
for row in range(9):
    row_formatted = []
    for col in range(9):
        row_formatted.append('{:.3f}'.format(move_probs[i]))
        i += 1
    print(' '.join(row_formatted))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
0.000 0.001 0.001 0.002 0.001 0.001 0.000 0.001 0.000
0.001 0.007 0.014 0.030 0.021 0.010 0.004 0.002 0.001
0.002 0.015 0.003 0.091 0.047 0.032 0.000 0.003 0.001
0.002 0.019 0.037 0.029 0.025 0.019 0.008 0.003 0.001
0.003 0.020 0.013 0.006 0.032 0.041 0.013 0.006 0.001
0.001 0.024 0.028 0.026 0.079 0.045 0.015 0.006 0.001
0.001 0.014 0.002 0.046 0.043 0.028 0.001 0.003 0.001
0.000 0.006 0.009 0.017 0.013 0.009 0.007 0.002 0.000
0.000 0.000 0.000 0.001 0.001 0.001 0.001 0.000 0.000


Starlight MacBook Air M2:
> Wall time: 1366.246495 seconds

> Test loss: 3.5951743125915527
> Test accuracy: 0.13778957724571228

MacBookPro M1 Pro:
> Wall time: 1680.059903 seconds

> Test loss: 3.5705721378326416
> Test accuracy: 0.1546814739704132

Seahawk Intel i7-4770 @ 3.4GHz AMD GPU:
> Wall time: 2068.261235 seconds

> Test loss: 3.589221715927124
> Test accuracy: 0.144305020570755

Carbon AMD Rizen 7 2700 8-Core NVIDIA GeForce RTX 4080 SUPER:
> Wall time: 326.127560 seconds

> Test loss: 3.5747978687286377
> Test accuracy: 0.14599420130252838

Steel:

> Wall time: 996.201202 seconds

> Test loss: 3.5864861011505127
> Test accuracy: 0.15202702581882477